# ZnTrack Parameters with dataclasses

To structure the parameters used in a Node it can be useful to pass them as a dataclass. The following Notebook will illustrate a small Example.

In [1]:
import pathlib

from zntrack import config

config.nb_name = "dataclasses_for_parameters.ipynb"

In [2]:
from zntrack.utils import cwd_temp_dir

temp_dir = cwd_temp_dir()

In [3]:
!git init
!dvc init

Initialized empty Git repository in /tmp/tmpumpig7qo/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [4]:
import dataclasses
from zntrack import Node, zn
import random

In [5]:
@dataclasses.dataclass
class Parameter:
    start: int
    stop: int
    step: int = 1


class ComputeRandomNumber(Node):
    parameter: Parameter = zn.Method()
    number = zn.outs()

    def __init__(self, parameter: Parameter = None, **kwargs):
        super().__init__(**kwargs)
        self.parameter = parameter

    def run(self):
        self.number = random.randrange(
            self.parameter.start, self.parameter.stop, self.parameter.step
        )

In [6]:
ComputeRandomNumber(parameter=Parameter(start=100, stop=200)).write_graph(no_exec=False)

2022-02-22 13:54:17,781 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.
2022-02-22 13:54:21,215 (WARNING): Running DVC command: 'dvc run -n ComputeRandomNumber ...'


In [7]:
print(ComputeRandomNumber.load().number)
print(ComputeRandomNumber.load().parameter)

146
Parameter(start=100, stop=200, step=1)


The arguments of the dataclass are saved in the `params.yaml` file and can also be modified there.

In [8]:
print(pathlib.Path("params.yaml").read_text())

ComputeRandomNumber:
    parameter:
        _cls: Parameter
        start: 100
        step: 1
        stop: 200

